# imports:

In [1]:
from os import listdir,mkdir,makedirs
from os.path import isfile, join, isdir,exists
import pandas as pd
import numpy as np
from scipy import stats
import re
import time
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from myplots import roundup, rounddown, find_decimal_fold, percentile_cut_off, rarefaction_calc, rarefaction_plot,draw_correlation_scatter
from matplotlib.ticker import FormatStrFormatter
import cPickle as pickle
from Bio.SeqUtils import GC
import seaborn as sns
import random
from scipy.stats import pearsonr,fisher_exact,ttest_ind,mannwhitneyu, sem
from skbio.diversity.alpha import shannon, simpson, berger_parker_d

from pop_organize import get_sample_data, get_sample_with_dfs
from SufficientStatistics.SufficientStatistics import *
from MyFunctionsShani import *
import math
from myplots import roundup, rounddown, find_decimal_fold
from skbio.stats.distance import mantel
from scipy.spatial.distance import braycurtis, pdist, euclidean


from GeneralFeaturePhenotypeInteractions.Feature_phenotype_functions import * 
from TCR_microbiome_interactions.TCR_microbiome_interactions_functions import *
from TCR_microbiome_interactions.TCR_microbiome_interactions_functions2 import *
from SampleLists.SampleFileFunctions import *
from PhenotypicData.PhenotypeGenerationFunctions import *
from CardioProject.CardioFunctions import *
from PredictionPipeline.PredictionFunctions import *
from TCR_feature_generation.SubsamplingFunctions import *

import os
from Utils import cacheOnDisk, Load
from SegalQueue.qp import qp,fakeqp
from addloglevels import sethandlers

#ML imports:
from xgboost import XGBClassifier
import lightgbm as lgb
from collections import OrderedDict
from sklearn.model_selection import GroupKFold,StratifiedKFold, KFold
import statsmodels.formula.api as sm
from sklearn.linear_model import LogisticRegression, LinearRegression
import shap
from sklearn import metrics, preprocessing
from sklearn.svm import LinearSVC
from sklearn.feature_selection import SelectFromModel,SelectKBest,chi2,mutual_info_classif,f_classif
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import *

MyPath='/net/mraid08/export/genie/Lab/Personal/ShaniBAF'

with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/PNP530','rb') as fp:
    PNP530=pickle.load(fp)
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/Cardio126','rb') as fp:
    Cardio126=pickle.load(fp)
PNP530Cardio126=PNP530+Cardio126

pd.set_option('display.expand_frame_repr', False)


done1
stop
stop
done1
stop
done
done
done
done
stop
stop
end of function!!!
end of function!!!


# definitions:

In [2]:
#####path definitions:
MyPath = '/net/mraid08/export/genie/Lab/Personal/ShaniBAF'

with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/PNP530', 'rb') as fp:
    PNP530 = pickle.load(fp)
with open('/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/Cardio126', 'rb') as fp:
    Cardio126 = pickle.load(fp)
PNP530Cardio126 = PNP530 + Cardio126

PRED_RESULTS_DIR='/net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/predictions2/'
FEATURES_DIR='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/Predictions/featureDFs/'
SAMPLE_LIST_DIR='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/Sample files/BD lists/'
CARDIO_PHEN_DIR='/net/mraid08/export/genie/Lab/Personal/ShaniBAF/TCR_real_data/CardioSamples/phenotypicData/'
CLUSTER_ANALYSIS_DIR='/net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/sharingAnalysis/seqClusters_allProd_maxdist1/'

NEW_FEATURES_DIR='/net/mraid08/export/jafar/Microbiome/Analyses/TCR/'

##### general definitions:
pd.set_option('display.expand_frame_repr', False)
pd.set_option('display.width', 1000)
np.set_printoptions(precision=4, linewidth=200)

cdate = str(time.strftime("%d%m%Y"))
cdate

'10042019'

# FOR TCR feature prediction - 5Mss with MB features

## species

In [50]:
#change for every run:

target_name='TCRfeatures'
task='reg'  ##'clas'/'reg'
nPreSelectedFeatures=50
isBinary=False
isGenotek=True
mother_pred_dir='TCR_features_updated'

taxLevel='Species' #'Species'/'Genus'
subsampling=5 #5/10/None



score_binclass='AUC'
score_reg='ExpVar'

minShared=0.05
maxShared=0.95
Ypath='/net/mraid08/export/jafar/Microbiome/Analyses/TCR/TCRfeatures/TCRfeaturesNoT_PCA10percShared_withRels_VDJ_noCorr0-999_nanFilled_noConsts_PNP530.xlsx'
mem_def=2

#do not change these arameters!
if task == 'clas': 
    score=score_binclass
    score_name=score
else: 
    score=None
    score_name=score_reg

minShared=str(minShared).replace('.','')
maxShared=str(maxShared).replace('.','')

if isBinary: isBinary='_binary'
else: isBinary=''
if isGenotek: isGenotek='genotek'
else: isGenotek='swab'

# print isBinary
# print isGenotek
    
XfileName=taxLevel+'%sMss' %subsampling+str(isGenotek)+str(isBinary)

output_dir=('/net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/predictions2/%(mother_pred_dir)s/%(target_name)s_XGB%(nPreSelectedFeatures)s\
_randomSearch_25_by%(XfileName)s_optBy%(score_name)s/' %{'mother_pred_dir':mother_pred_dir,'target_name':target_name,'nPreSelectedFeatures':nPreSelectedFeatures,
                                                        'XfileName':XfileName, 'score_name':score_name})
Xpath=('/net/mraid08/export/jafar/Microbiome/Analyses/TCR/Mb_data_by_BD/\
LargeOrNewGenusSGBs_5MSubsampling_00001threshold_Noneratio_sSGB_byBD_PNP530_%(minShared)s_%(maxShared)s\
%(isBinary)s_%(isGenotek)s_withFeatures.xlsx' %{'minShared':minShared,'maxShared':maxShared,'isBinary':isBinary,'isGenotek':isGenotek})

pathToModelParams='/net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/predictions2/Model_params_files/XGB_model_params_1.xlsx'

command = ('python /home/sbenari/workspace/Microbiome/ShaniBA/PredictionPipeline/PredictionModuleShani.py %(output_dir)s \
-path_to_X %(Xpath)s -path_to_Y %(Ypath)s -parameter_search_type randomSearch -n_random 25 -model XGB \
-pathToModelParams %(pathToModelParams)s -mem_def %(mem_def)s -n_cols_per_job 1 -nPreSelectedFeatures %(nPreSelectedFeatures)s'\
%{'output_dir': output_dir, 'Xpath':Xpath, 'Ypath': Ypath, 'pathToModelParams':pathToModelParams, 'nPreSelectedFeatures':nPreSelectedFeatures,
   'pathToModelParams':pathToModelParams, 'mem_def':mem_def})

print command

python /home/sbenari/workspace/Microbiome/ShaniBA/PredictionPipeline/PredictionModuleShani.py /net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/predictions2/TCR_features_updated/TCRfeatures_XGB50_randomSearch_25_bySpecies5Mssgenotek_optByExpVar/ -path_to_X /net/mraid08/export/jafar/Microbiome/Analyses/TCR/Mb_data_by_BD/LargeOrNewGenusSGBs_5MSubsampling_00001threshold_Noneratio_sSGB_byBD_PNP530_005_095_genotek_withFeatures.xlsx -path_to_Y /net/mraid08/export/jafar/Microbiome/Analyses/TCR/TCRfeatures/TCRfeaturesNoT_PCA10percShared_withRels_VDJ_noCorr0-999_nanFilled_noConsts_PNP530.xlsx -parameter_search_type randomSearch -n_random 25 -model XGB -pathToModelParams /net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/predictions2/Model_params_files/XGB_model_params_1.xlsx -mem_def 2 -n_cols_per_job 1 -nPreSelectedFeatures 50


## genus

In [57]:
#change for every run:

target_name='TCRfeatures'
task='reg'  ##'clas'/'reg'
nPreSelectedFeatures=99999
isBinary=False
isGenotek=True
mother_pred_dir='TCR_features_updated/byMBgenus'

taxLevel='Genus' #'Species'/'Genus'
subsampling=5 #5/10/None



score_binclass='AUC'
score_reg='ExpVar'

minShared=0.05
maxShared=0.95
Ypath='/net/mraid08/export/jafar/Microbiome/Analyses/TCR/TCRfeatures/TCRfeaturesNoT_PCA10percShared_withRels_VDJ_noCorr0-999_nanFilled_noConsts_PNP530.xlsx'
mem_def=2

#do not change these arameters!
if task == 'clas': 
    score=score_binclass
    score_name=score
else: 
    score=None
    score_name=score_reg

minShared=str(minShared).replace('.','')
maxShared=str(maxShared).replace('.','')

if isBinary: isBinary='_binary'
else: isBinary=''
if isGenotek: isGenotek='genotek'
else: isGenotek='swab'

# print isBinary
# print isGenotek
    
XfileName=taxLevel+'%sMss' %subsampling+str(isGenotek)+str(isBinary)

output_dir=('/net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/predictions2/%(mother_pred_dir)s/%(target_name)s_XGB%(nPreSelectedFeatures)s\
_randomSearch_25_by%(XfileName)s_optBy%(score_name)s/' %{'mother_pred_dir':mother_pred_dir,'target_name':target_name,'nPreSelectedFeatures':nPreSelectedFeatures,
                                                        'XfileName':XfileName, 'score_name':score_name})
Xpath=('/net/mraid08/export/jafar/Microbiome/Analyses/TCR/Mb_data_by_BD/\
LargeOrNewGenusSGBs_5MSubsampling_00001threshold_Noneratio_gSGB_byBD_PNP530_%(minShared)s_%(maxShared)s\
%(isBinary)s_%(isGenotek)s_withFeatures.xlsx' %{'minShared':minShared,'maxShared':maxShared,'isBinary':isBinary,'isGenotek':isGenotek})

pathToModelParams='/net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/predictions2/Model_params_files/XGB_model_params_1.xlsx'

command = ('python /home/sbenari/workspace/Microbiome/ShaniBA/PredictionPipeline/PredictionModuleShani.py %(output_dir)s \
-path_to_X %(Xpath)s -path_to_Y %(Ypath)s -parameter_search_type randomSearch -n_random 25 -model XGB \
-pathToModelParams %(pathToModelParams)s -mem_def %(mem_def)s -n_cols_per_job 1 -nPreSelectedFeatures %(nPreSelectedFeatures)s'\
%{'output_dir': output_dir, 'Xpath':Xpath, 'Ypath': Ypath, 'pathToModelParams':pathToModelParams, 'nPreSelectedFeatures':nPreSelectedFeatures,
   'pathToModelParams':pathToModelParams, 'mem_def':mem_def})

print command

python /home/sbenari/workspace/Microbiome/ShaniBA/PredictionPipeline/PredictionModuleShani.py /net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/predictions2/TCR_features_updated/byMBgenus/TCRfeatures_XGB99999_randomSearch_25_byGenus5Mssgenotek_optByExpVar/ -path_to_X /net/mraid08/export/jafar/Microbiome/Analyses/TCR/Mb_data_by_BD/LargeOrNewGenusSGBs_5MSubsampling_00001threshold_Noneratio_gSGB_byBD_PNP530_005_095_genotek_withFeatures.xlsx -path_to_Y /net/mraid08/export/jafar/Microbiome/Analyses/TCR/TCRfeatures/TCRfeaturesNoT_PCA10percShared_withRels_VDJ_noCorr0-999_nanFilled_noConsts_PNP530.xlsx -parameter_search_type randomSearch -n_random 25 -model XGB -pathToModelParams /net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/predictions2/Model_params_files/XGB_model_params_1.xlsx -mem_def 2 -n_cols_per_job 1 -nPreSelectedFeatures 99999


# FOR top100 clusters binary prediction - 5Mss,  with MB features

## SPECIES:

In [52]:
#change for every run:

target_name='top100Clusters'
task='reg'  ##'clas'/'reg'
nPreSelectedFeatures=50
isBinary=False
isGenotek=True
mother_pred_dir='TCR_seqs_clusters'

taxLevel='Species' #'Species'/'Genus'
subsampling=5 #5/10/None



score_binclass='AUC'
score_reg='ExpVar'

minShared=0.05
maxShared=0.95
Ypath='/net/mraid08/export/jafar/Microbiome/Analyses/TCR/TCR_seqs_clusters/top100_clusterDF_PNP530_binary.xlsx'
mem_def=2

#do not change these arameters!
if task == 'clas': 
    score=score_binclass
    score_name=score
else: 
    score=None
    score_name=score_reg

minShared=str(minShared).replace('.','')
maxShared=str(maxShared).replace('.','')

if isBinary: isBinary='_binary'
else: isBinary=''
if isGenotek: isGenotek='genotek'
else: isGenotek='swab'

# print isBinary
# print isGenotek
    
XfileName=taxLevel+'%sMss' %subsampling+str(isGenotek)+str(isBinary)

output_dir=('/net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/predictions2/%(mother_pred_dir)s/%(target_name)s_XGB%(nPreSelectedFeatures)s\
_randomSearch_25_by%(XfileName)s_optBy%(score_name)s/' %{'mother_pred_dir':mother_pred_dir,'target_name':target_name,'nPreSelectedFeatures':nPreSelectedFeatures,
                                                        'XfileName':XfileName, 'score_name':score_name})
Xpath=('/net/mraid08/export/jafar/Microbiome/Analyses/TCR/Mb_data_by_BD/\
LargeOrNewGenusSGBs_5MSubsampling_00001threshold_Noneratio_sSGB_byBD_PNP530_%(minShared)s_%(maxShared)s\
%(isBinary)s_%(isGenotek)s_withFeatures.xlsx' %{'minShared':minShared,'maxShared':maxShared,'isBinary':isBinary,'isGenotek':isGenotek})

pathToModelParams='/net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/predictions2/Model_params_files/XGB_model_params_1.xlsx'

command = ('python /home/sbenari/workspace/Microbiome/ShaniBA/PredictionPipeline/PredictionModuleShani.py %(output_dir)s \
-path_to_X %(Xpath)s -path_to_Y %(Ypath)s -parameter_search_type randomSearch -n_random 25 -model XGB \
-pathToModelParams %(pathToModelParams)s -mem_def %(mem_def)s -n_cols_per_job 1 -nPreSelectedFeatures %(nPreSelectedFeatures)s'\
%{'output_dir': output_dir, 'Xpath':Xpath, 'Ypath': Ypath, 'pathToModelParams':pathToModelParams, 'nPreSelectedFeatures':nPreSelectedFeatures,
   'pathToModelParams':pathToModelParams, 'mem_def':mem_def})

print command

python /home/sbenari/workspace/Microbiome/ShaniBA/PredictionPipeline/PredictionModuleShani.py /net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/predictions2/TCR_seqs_clusters/top100Clusters_XGB50_randomSearch_25_bySpecies5Mssgenotek_optByExpVar/ -path_to_X /net/mraid08/export/jafar/Microbiome/Analyses/TCR/Mb_data_by_BD/LargeOrNewGenusSGBs_5MSubsampling_00001threshold_Noneratio_sSGB_byBD_PNP530_005_095_genotek_withFeatures.xlsx -path_to_Y /net/mraid08/export/jafar/Microbiome/Analyses/TCR/TCR_seqs_clusters/top100_clusterDF_PNP530_binary.xlsx -parameter_search_type randomSearch -n_random 25 -model XGB -pathToModelParams /net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/predictions2/Model_params_files/XGB_model_params_1.xlsx -mem_def 2 -n_cols_per_job 1 -nPreSelectedFeatures 50


## GEUNS:

In [53]:
#change for every run:

target_name='top100Clusters'
task='reg'  ##'clas'/'reg'
nPreSelectedFeatures=50
isBinary=False
isGenotek=False
mother_pred_dir='TCR_seqs_clusters/byMBgenus'

taxLevel='Genus' #'Species'/'Genus'
subsampling=5 #5/10/None



score_binclass='AUC'
score_reg='ExpVar'

minShared=0.05
maxShared=0.95
Ypath='/net/mraid08/export/jafar/Microbiome/Analyses/TCR/TCR_seqs_clusters/top100_clusterDF_PNP530_binary.xlsx'
mem_def=2



#do not change these arameters!
if task == 'clas': 
    score=score_binclass
    score_name=score
else: 
    score=None
    score_name=score_reg

minShared=str(minShared).replace('.','')
maxShared=str(maxShared).replace('.','')

if isBinary: isBinary='_binary'
else: isBinary=''
if isGenotek: isGenotek='genotek'
else: isGenotek='swab'

# print isBinary
# print isGenotek
    
XfileName=taxLevel+'%sMss' %subsampling+str(isGenotek)+str(isBinary)

output_dir=('/net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/predictions2/%(mother_pred_dir)s/%(target_name)s_XGB%(nPreSelectedFeatures)s\
_randomSearch_25_by%(XfileName)s_optBy%(score_name)s/' %{'mother_pred_dir':mother_pred_dir,'target_name':target_name,'nPreSelectedFeatures':nPreSelectedFeatures,
                                                        'XfileName':XfileName, 'score_name':score_name})
Xpath=('/net/mraid08/export/jafar/Microbiome/Analyses/TCR/Mb_data_by_BD/\
LargeOrNewGenusSGBs_5MSubsampling_00001threshold_Noneratio_gSGB_byBD_PNP530_%(minShared)s_%(maxShared)s\
%(isBinary)s_%(isGenotek)s_withFeatures.xlsx' %{'minShared':minShared,'maxShared':maxShared,'isBinary':isBinary,'isGenotek':isGenotek})

pathToModelParams='/net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/predictions2/Model_params_files/XGB_model_params_1.xlsx'

command = ('python /home/sbenari/workspace/Microbiome/ShaniBA/PredictionPipeline/PredictionModuleShani.py %(output_dir)s \
-path_to_X %(Xpath)s -path_to_Y %(Ypath)s -parameter_search_type randomSearch -n_random 25 -model XGB \
-pathToModelParams %(pathToModelParams)s -mem_def %(mem_def)s -n_cols_per_job 1 -nPreSelectedFeatures %(nPreSelectedFeatures)s'\
%{'output_dir': output_dir, 'Xpath':Xpath, 'Ypath': Ypath, 'pathToModelParams':pathToModelParams, 'nPreSelectedFeatures':nPreSelectedFeatures,
   'pathToModelParams':pathToModelParams, 'mem_def':mem_def})

print command

python /home/sbenari/workspace/Microbiome/ShaniBA/PredictionPipeline/PredictionModuleShani.py /net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/predictions2/TCR_seqs_clusters/byMBgenus/top100Clusters_XGB50_randomSearch_25_byGenus5Mssswab_optByExpVar/ -path_to_X /net/mraid08/export/jafar/Microbiome/Analyses/TCR/Mb_data_by_BD/LargeOrNewGenusSGBs_5MSubsampling_00001threshold_Noneratio_gSGB_byBD_PNP530_005_095_swab_withFeatures.xlsx -path_to_Y /net/mraid08/export/jafar/Microbiome/Analyses/TCR/TCR_seqs_clusters/top100_clusterDF_PNP530_binary.xlsx -parameter_search_type randomSearch -n_random 25 -model XGB -pathToModelParams /net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/predictions2/Model_params_files/XGB_model_params_1.xlsx -mem_def 2 -n_cols_per_job 1 -nPreSelectedFeatures 50


# predict top20isCardio features:

## by species

In [59]:
#change for every run:

target_name='top100Clusters'
task='reg'  ##'clas'/'reg'
nPreSelectedFeatures=50
isBinary=True
isGenotek=True
mother_pred_dir='TCR_features_updated/top20isCardioShapFeatures'

taxLevel='Species' #'Species'/'Genus'
subsampling=5 #5/10/None



score_binclass='AUC'
score_reg='ExpVar'

minShared=0.05
maxShared=0.95
Ypath='/net/mraid08/export/jafar/Microbiome/Analyses/TCR/TCRfeatures/TCRfeaturesNoT_PCA10percShared_withRels_VDJ_noCorr0-999_nanFilled_noConsts_PNP530_top20isCardioShapFeatures.xlsx'
mem_def=2



#do not change these arameters!
if task == 'clas': 
    score=score_binclass
    score_name=score
else: 
    score=None
    score_name=score_reg

minShared=str(minShared).replace('.','')
maxShared=str(maxShared).replace('.','')

if isBinary: isBinary='_binary'
else: isBinary=''
if isGenotek: isGenotek='genotek'
else: isGenotek='swab'

# print isBinary
# print isGenotek
    
XfileName=taxLevel+'%sMss' %subsampling+str(isGenotek)+str(isBinary)

output_dir=('/net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/predictions2/%(mother_pred_dir)s/%(target_name)s_XGB%(nPreSelectedFeatures)s\
_randomSearch_25_by%(XfileName)s_optBy%(score_name)s/' %{'mother_pred_dir':mother_pred_dir,'target_name':target_name,'nPreSelectedFeatures':nPreSelectedFeatures,
                                                        'XfileName':XfileName, 'score_name':score_name})
Xpath=('/net/mraid08/export/jafar/Microbiome/Analyses/TCR/Mb_data_by_BD/\
LargeOrNewGenusSGBs_5MSubsampling_00001threshold_Noneratio_sSGB_byBD_PNP530_%(minShared)s_%(maxShared)s\
%(isBinary)s_%(isGenotek)s_withFeatures.xlsx' %{'minShared':minShared,'maxShared':maxShared,'isBinary':isBinary,'isGenotek':isGenotek})

pathToModelParams='/net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/predictions2/Model_params_files/XGB_model_params_1.xlsx'

command = ('python /home/sbenari/workspace/Microbiome/ShaniBA/PredictionPipeline/PredictionModuleShani.py %(output_dir)s \
-path_to_X %(Xpath)s -path_to_Y %(Ypath)s -parameter_search_type randomSearch -n_random 25 -model XGB \
-pathToModelParams %(pathToModelParams)s -mem_def %(mem_def)s -n_cols_per_job 1 -nPreSelectedFeatures %(nPreSelectedFeatures)s'\
%{'output_dir': output_dir, 'Xpath':Xpath, 'Ypath': Ypath, 'pathToModelParams':pathToModelParams, 'nPreSelectedFeatures':nPreSelectedFeatures,
   'pathToModelParams':pathToModelParams, 'mem_def':mem_def})

print command

python /home/sbenari/workspace/Microbiome/ShaniBA/PredictionPipeline/PredictionModuleShani.py /net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/predictions2/TCR_features_updated/top20isCardioShapFeatures/top100Clusters_XGB50_randomSearch_25_bySpecies5Mssgenotek_binary_optByExpVar/ -path_to_X /net/mraid08/export/jafar/Microbiome/Analyses/TCR/Mb_data_by_BD/LargeOrNewGenusSGBs_5MSubsampling_00001threshold_Noneratio_sSGB_byBD_PNP530_005_095_binary_genotek_withFeatures.xlsx -path_to_Y /net/mraid08/export/jafar/Microbiome/Analyses/TCR/TCRfeatures/TCRfeaturesNoT_PCA10percShared_withRels_VDJ_noCorr0-999_nanFilled_noConsts_PNP530_top20isCardioShapFeatures.xlsx -parameter_search_type randomSearch -n_random 25 -model XGB -pathToModelParams /net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/predictions2/Model_params_files/XGB_model_params_1.xlsx -mem_def 2 -n_cols_per_job 1 -nPreSelectedFeatures 50


## by genus

In [61]:
#change for every run:

target_name='top100Clusters'
task='reg'  ##'clas'/'reg'
nPreSelectedFeatures=50
isBinary=False
isGenotek=True
mother_pred_dir='TCR_features_updated/top20isCardioShapFeatures'

taxLevel='Genus' #'Species'/'Genus'
subsampling=5 #5/10/None



score_binclass='AUC'
score_reg='ExpVar'

minShared=0.05
maxShared=0.95
Ypath='/net/mraid08/export/jafar/Microbiome/Analyses/TCR/TCRfeatures/TCRfeaturesNoT_PCA10percShared_withRels_VDJ_noCorr0-999_nanFilled_noConsts_PNP530_top20isCardioShapFeatures.xlsx'
mem_def=2



#do not change these arameters!
if task == 'clas': 
    score=score_binclass
    score_name=score
else: 
    score=None
    score_name=score_reg

minShared=str(minShared).replace('.','')
maxShared=str(maxShared).replace('.','')

if isBinary: isBinary='_binary'
else: isBinary=''
if isGenotek: isGenotek='genotek'
else: isGenotek='swab'

# print isBinary
# print isGenotek
    
XfileName=taxLevel+'%sMss' %subsampling+str(isGenotek)+str(isBinary)

output_dir=('/net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/predictions2/%(mother_pred_dir)s/%(target_name)s_XGB%(nPreSelectedFeatures)s\
_randomSearch_25_by%(XfileName)s_optBy%(score_name)s/' %{'mother_pred_dir':mother_pred_dir,'target_name':target_name,'nPreSelectedFeatures':nPreSelectedFeatures,
                                                        'XfileName':XfileName, 'score_name':score_name})
Xpath=('/net/mraid08/export/jafar/Microbiome/Analyses/TCR/Mb_data_by_BD/\
LargeOrNewGenusSGBs_5MSubsampling_00001threshold_Noneratio_gSGB_byBD_PNP530_%(minShared)s_%(maxShared)s\
%(isBinary)s_%(isGenotek)s_withFeatures.xlsx' %{'minShared':minShared,'maxShared':maxShared,'isBinary':isBinary,'isGenotek':isGenotek})

pathToModelParams='/net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/predictions2/Model_params_files/XGB_model_params_1.xlsx'

command = ('python /home/sbenari/workspace/Microbiome/ShaniBA/PredictionPipeline/PredictionModuleShani.py %(output_dir)s \
-path_to_X %(Xpath)s -path_to_Y %(Ypath)s -parameter_search_type randomSearch -n_random 25 -model XGB \
-pathToModelParams %(pathToModelParams)s -mem_def %(mem_def)s -n_cols_per_job 1 -nPreSelectedFeatures %(nPreSelectedFeatures)s'\
%{'output_dir': output_dir, 'Xpath':Xpath, 'Ypath': Ypath, 'pathToModelParams':pathToModelParams, 'nPreSelectedFeatures':nPreSelectedFeatures,
   'pathToModelParams':pathToModelParams, 'mem_def':mem_def})

print command

python /home/sbenari/workspace/Microbiome/ShaniBA/PredictionPipeline/PredictionModuleShani.py /net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/predictions2/TCR_features_updated/top20isCardioShapFeatures/top100Clusters_XGB50_randomSearch_25_byGenus5Mssgenotek_optByExpVar/ -path_to_X /net/mraid08/export/jafar/Microbiome/Analyses/TCR/Mb_data_by_BD/LargeOrNewGenusSGBs_5MSubsampling_00001threshold_Noneratio_gSGB_byBD_PNP530_005_095_genotek_withFeatures.xlsx -path_to_Y /net/mraid08/export/jafar/Microbiome/Analyses/TCR/TCRfeatures/TCRfeaturesNoT_PCA10percShared_withRels_VDJ_noCorr0-999_nanFilled_noConsts_PNP530_top20isCardioShapFeatures.xlsx -parameter_search_type randomSearch -n_random 25 -model XGB -pathToModelParams /net/mraid08/export/jafar/Microbiome/Analyses/ShaniBAF/predictions2/Model_params_files/XGB_model_params_1.xlsx -mem_def 2 -n_cols_per_job 1 -nPreSelectedFeatures 50
